In [1]:
import requests, json
from pprint import pprint

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 8224426}


### Players

In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'chance_of_playing_next_round': 0,
 'chance_of_playing_this_round': 0,
 'clean_sheets': 0,
 'code': 37605,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -3,
 'cost_change_start_fall': 3,
 'creativity': '0.0',
 'creativity_rank': 703,
 'creativity_rank_type': 288,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 3,
 'ep_next': '0.0',
 'ep_this': '0.0',
 'event_points': 0,
 'first_name': 'Mesut',
 'form': '0.0',
 'goals_conceded': 0,
 'goals_scored': 0,
 'ict_index': '0.0',
 'ict_index_rank': 703,
 'ict_index_rank_type': 288,
 'id': 1,
 'in_dreamteam': False,
 'influence': '0.0',
 'influence_rank': 703,
 'influence_rank_type': 288,
 'minutes': 0,
 'news': "Not included in Arsenal's 25-man Premier League squad",
 'news_added': '2020-10-20T22:30:18.118477Z',
 'now_cost': 67,
 '

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Özil,1,3
1,2,Sokratis,1,2
2,3,David Luiz,1,2
3,4,Aubameyang,1,3
4,5,Cédric,1,2


In [6]:
data_folder = 'data/2020-21/'

# select columns of interest and save as CSV
players = players[
    [
        'first_name',
        'second_name',
        'web_name',
        'id',
        'team',
        'total_points',
        'dreamteam_count',
        'element_type',
        'in_dreamteam',
        'now_cost',
        'points_per_game',
        'minutes',
        'goals_scored',
        'assists',
        'clean_sheets',
        'goals_conceded',
        'own_goals',
        'penalties_saved',
        'penalties_missed',
        'yellow_cards',
        'red_cards',
        'saves',
        'bonus',
        'bps',
        'influence',
        'creativity',
        'threat',
        'ict_index',
        'influence_rank',
        'influence_rank_type',
        'creativity_rank',
        'creativity_rank_type',
        'threat_rank',
        'threat_rank_type',
        'ict_index_rank',
        'ict_index_rank_type'
    ]
]

players.to_csv(data_folder + 'players.csv', index=False)

### Teams

In [7]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1200,1250,1130,1150,1220,1210,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1100,1160,1120,1140,1080,1100,2
2,36,0,None,3,0,Brighton,0,0,0,BHA,3,None,False,0,1130,1130,1150,1190,1100,1130,131
3,90,0,None,4,0,Burnley,0,0,0,BUR,2,None,False,0,1060,1070,1040,1090,1040,1080,43
4,8,0,None,5,0,Chelsea,0,0,0,CHE,4,None,False,0,1250,1280,1260,1300,1250,1310,4


In [8]:
# select columns of interest and save as CSV
teams.drop(['code', 'draw', 'form', 'loss', 'points', 'position',
            'team_division', 'unavailable', 'pulse_id'], axis=1, inplace=True)

teams.to_csv(data_folder + 'teams.csv', index=False)

### Player positions

In [9]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],80
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],244
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],288
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],91


In [10]:
# select columns of interest and save as CSV
positions = positions[['id', 'singular_name', 'singular_name_short']]
positions.to_csv(data_folder + 'positions.csv', index=False)

### Player gameweek history

In [11]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [12]:
pprint(r['history'][0])

{'assists': 0,
 'bonus': 0,
 'bps': 19,
 'clean_sheets': 1,
 'creativity': '15.3',
 'element': 4,
 'fixture': 2,
 'goals_conceded': 0,
 'goals_scored': 1,
 'ict_index': '10.6',
 'influence': '36.6',
 'kickoff_time': '2020-09-12T11:30:00Z',
 'minutes': 90,
 'opponent_team': 8,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'red_cards': 0,
 'round': 1,
 'saves': 0,
 'selected': 2823465,
 'team_a_score': 3,
 'team_h_score': 0,
 'threat': '54.0',
 'total_points': 7,
 'transfers_balance': 0,
 'transfers_in': 0,
 'transfers_out': 0,
 'value': 120,
 'was_home': False,
 'yellow_cards': 1}


In [13]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df


# show player #4's gameweek history
get_gameweek_history(4)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head()

,round,total_points,minutes,goals_scored,assists
0,1,7,90,1,0
1,2,5,90,0,1
2,3,2,90,0,0
3,4,2,90,0,0
4,5,2,90,0,0


In [24]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    df.insert(0, 'id', player_id)
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'id',
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,id,season_name,total_points,minutes,goals_scored,assists
0,1,2013/14,137,2141,5,10
1,1,2014/15,103,1857,4,6
2,1,2015/16,200,3036,6,19
3,1,2016/17,167,2841,8,11
4,1,2017/18,112,2161,4,9
5,1,2018/19,89,1732,5,3
6,1,2019/20,53,1439,1,3


## Combined dataset

In [15]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Mesut,Özil,Özil,Arsenal,MID
1,4,Pierre-Emerick,Aubameyang,Aubameyang,Arsenal,MID
2,9,Granit,Xhaka,Xhaka,Arsenal,MID
3,18,Nicolas,Pépé,Pépé,Arsenal,MID
4,19,Lucas,Torreira,Torreira,Arsenal,MID


In [16]:
from tqdm.auto import tqdm
tqdm.pandas()

In [17]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# rename columns
points.rename({'element':'id'}, axis=1, inplace=True)

# save to CSV
points.to_csv(data_folder + 'gameweek_history.csv', index=False)

In [25]:
# get season histories for each player
seasons = players['id_player'].progress_apply(get_season_history)

# combine results into single dataframe
seasons = pd.concat(df for df in seasons)

# rename columns
points.rename({'element':'id'}, axis=1, inplace=True)

# save to CSV
seasons.to_csv(data_folder + 'seasons_history.csv', index=False)